In [ ]:
import librosa
import numpy as np
from IPython.display import Audio
import os

In [ ]:
positive_sample_path = '/mnt/disks/msd/audio/A/A/A/TRAAAAW128F429D538.mp3'
y, sr = librosa.load(positive_sample_path, sr=None)
Audio(data=y, rate=sr)

In [ ]:
audio_file_data = y[44100*5:44100*15]
Audio(audio_file_data, rate=sr)

In [ ]:
#alisaing

# Define the time shift (in samples)
shift = 5000

# Shift different parts of the signal in time
scrambled_signal = np.concatenate((audio_file_data[shift:], audio_file_data[:shift]))

# Play and display the scrambled signal
Audio(scrambled_signal, rate=sr)

In [ ]:
# Define the noise level (in dB)
noise_level = -20

# Generate noise with the same length as the signal
noise = np.random.normal(scale=np.sqrt(10**(noise_level/10)), size=len(audio_file_data))

# Add the noise to the signal
scrambled_signal = audio_file_data + noise

# Compute the SNR
signal_power = np.sum(audio_file_data**2) / len(audio_file_data)
noise_power = np.sum(noise**2) / len(noise)
snr = 10 * np.log10(signal_power / noise_power)

print("SNR:", snr)

# Check if SNR is greater than threshold n
n = 20  # set threshold
if snr > n:
    raise ValueError("Signal is not musically perceivable due to high noise level.")
else:
  pass

# Play and display the scrambled signal
Audio(scrambled_signal, rate=sr)

In [ ]:
from audio_augmentations import *

audio, sr = torchaudio.load("tests/classical.00002.wav")

num_samples = sr * 5
transforms = [
    RandomResizedCrop(n_samples=num_samples),
    RandomApply([PolarityInversion()], p=0.8),
    RandomApply([Noise(min_snr=0.001, max_snr=0.005)], p=0.3),
    RandomApply([Gain()], p=0.2),
    HighLowPass(sample_rate=sr), # this augmentation will always be applied in this aumgentation chain!
    RandomApply([Delay(sample_rate=sr)], p=0.5),
    RandomApply([PitchShift(
        n_samples=num_samples,
        sample_rate=sr
    )], p=0.4),
    RandomApply([Reverb(sample_rate=sr)], p=0.3)
]